### “We manage what we measure, but frequently we measure what is easy.” - Datanuts meeting

link to dataset
https://www.kaggle.com/lewisgmorris/warehouse-picking-times

Content

PH_DOC - unique order reference

PH_SORDER - unique order reference

PH_DELIVER - unique delivery reference

PH_PICKEDB - name of picker

PH_PICKSTA - datetime of picking started

PH_PICKEND - datetime of picking ended

PH_TOTALLI - total lines picked

PH_TOTALBO - total boxes used

Inspiration

Can you find the patterns in the numbers?

Suggestions:

What days are busiest?   
Are things getting better or worse?   
Who is the best?   
Whos not pulling their weight?   

#### As seen above these are typical questions asked from UPH (units per hour) information. But is this the whole story?     
Assuming that a worker is not "pulling their weight" because of low UPH assumes that people generally don't want to perform well and puts this burden on the individual to fix the problem. This removes the responsibility from management to find bottle necks and other sources of delay that prevent workers from succeeding and also allows senior managers to erroniously compare disperate environments.

#### What if we assumed that every worker wants to succeed and limitations in the environment prevents them from doing so?    
This mindset would challenge managers to find and fix the areas and processes that are getting in the way of success and/or to determine if the cost of fixing the issue is more or less expensive than hiring additional labor and would provide a more accurate basis of performance assessment.

#### How do we find the bottlenecks and issues that prevent success?    
Ask the people doing the job!

#### Some frequent limiting factors that are not typically considered when establishing or comparing UPH metrics from site to site:    
- Bulk/Lumber distribution: weather, % of warehouse indoor/outdoor, ground level/ramp, state truck maximum weight capacity, product diversity of size, product diversity of weight, max forklift capcity, average drive distance from stacks to truck, asile width of main travel paths, max safe forklift operating speed, average # years operator experience, average # units/truck, % live unload vs staged trucks, lighting levels, equipment down time, in stock %, varibility in product count/package size, average temperature

- Indoor operations/non-perishable: product diversity of size, product diversity of weight, max forklift capcity, average drive distance from stacks to truck, asile width of main travel paths, max safe forklift operating speed, average # years operator experience, average # units/truck, % live unload vs staged trucks, floor load/pallet, % full pallet items/individual pick, conveyer lines/pallet jack/forklift pick, products with team lift requirements, sqft of pick zone, average # picks per order, equipment down time, in stock %, varibility in product count/package size, average temperature

- Store pulling: store volume, store sqft, product diversity of size, product diversity of weight, order storage area(walk-in/transfer to shelf), sqft of pick zone, average # picks per order, length of shift, task distribution among operators, operator experience(and/or time for response on questions), in stock %, varibility in product count/package size, % use best judgement/do not substitue orders

#### Some frequent limiting factors that are not typically considered when establishing or comparing UPH metrics within a site:
- Bulk/Lumber distribution: average # years operator experience, average # units/truck, % live unload vs staged trucks, task distribution among operators (inbound, outbound, "non-productive"(safety walks, meetings, training), average # units/truck, % live unload vs staged trucks, equipment down time, in stock %, varibility in product count/package size, max forklift capcity, average drive distance from stacks to truck

- Indoor operations/non-perishable: product diversity of size, product diversity of weight, max forklift capcity, average drive distance from stacks to truck, asile width of main travel paths, average # years operator experience, average # units/truck, % live unload vs staged trucks, floor load/pallet, % full pallet items/individual pick, conveyer lines/pallet jack/forklift pick, % products with team lift requirements, sqft of pick zone, average # picks per order, equipment down time, in stock %, varibility in product count/package size

- Store pulling: sproduct diversity of size, product diversity of weight, sqft of pick zone, average # picks per order, length of shift, task distribution among operators, operator experience(and/or time for response on questions), in stock %, % use best judgement/do not substitue orders

Ideally I would find data for all of the above information then explore to see which information showed the greatest impact for predicting units per hour and could build a ML model to predict UPH for a specific site or within a site.

Data source ideas:
Bulk/Lumber distribution: find my old files, maybe reach out to previous connections to see if any info could be exported

Create a fictional warehouse based off of this picking dataset. Use public retail sales dataset for a product mix and generate random info within a range for product weight, size. Use apriori algoithm to generate product frequencies and create fictional orders?



In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('pick data.csv', low_memory=False)

In [3]:
df.shape

(159980, 13)

In [4]:
df.columns

Index(['Id', 'PH_DOC', 'PH_SORDER', 'PH_DELIVER', 'PH_PICKEDB', 'PH_PICKSTA',
       'PH_PICKEND', 'PH_TOTALLI', 'PH_TOTALBO', 'Unnamed: 9', 'Unnamed: 10',
       'Unnamed: 11', 'Unnamed: 12'],
      dtype='object')

In [5]:
df.isna().sum()

Id                  0
PH_DOC              0
PH_SORDER           0
PH_DELIVER     122907
PH_PICKEDB         14
PH_PICKSTA         13
PH_PICKEND          1
PH_TOTALLI          0
PH_TOTALBO          0
Unnamed: 9     159980
Unnamed: 10    159980
Unnamed: 11    159980
Unnamed: 12    159980
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159980 entries, 0 to 159979
Data columns (total 13 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Id           159980 non-null  int64  
 1   PH_DOC       159980 non-null  object 
 2   PH_SORDER    159980 non-null  object 
 3   PH_DELIVER   37073 non-null   object 
 4   PH_PICKEDB   159966 non-null  object 
 5   PH_PICKSTA   159967 non-null  object 
 6   PH_PICKEND   159979 non-null  object 
 7   PH_TOTALLI   159980 non-null  int64  
 8   PH_TOTALBO   159980 non-null  int64  
 9   Unnamed: 9   0 non-null       float64
 10  Unnamed: 10  0 non-null       float64
 11  Unnamed: 11  0 non-null       float64
 12  Unnamed: 12  0 non-null       float64
dtypes: float64(4), int64(3), object(6)
memory usage: 15.9+ MB


NOTE: drop all Unnamed columns, these contain only null values

In [7]:
# create df of useable columns only, PH_Doc and PH_sorder are showing 'value#' for most entries, 
# PH_deliver contains large # of nulls

pickdf = df[['PH_PICKEDB', 'PH_PICKSTA', 'PH_PICKEND', 'PH_TOTALLI', 'PH_TOTALBO']]

In [8]:
pickdf.head()

,PH_PICKEDB,PH_PICKSTA,PH_PICKEND,PH_TOTALLI,PH_TOTALBO
0,PAUL,23/02/2015 12:25:47,23/02/2015 12:25:51,2,1
1,PAUL,23/02/2015 13:48:03,23/02/2015 13:48:05,1,1
2,PAUL,23/02/2015 14:18:13,23/02/2015 14:18:14,2,1
3,PAUL,23/02/2015 14:27:52,23/02/2015 14:29:55,2,1
4,LEWIS,26/02/2015 11:38:22,26/02/2015 11:39:27,3,1


In [9]:
# write this df to csv for future use
#pickdf.to_csv('pickdf.csv')

In [10]:
# Wrangling
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# Exploring
import scipy.stats as stats

# Visualizing
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.model_selection import learning_curve

pd.options.display.float_format = '{:20,.2f}'.format

In [11]:
pickdf = pd.read_csv('pickdf.csv', index_col=0)

In [12]:
pickdf.head()

,PH_PICKEDB,PH_PICKSTA,PH_PICKEND,PH_TOTALLI,PH_TOTALBO
0,PAUL,23/02/2015 12:25:47,23/02/2015 12:25:51,2,1
1,PAUL,23/02/2015 13:48:03,23/02/2015 13:48:05,1,1
2,PAUL,23/02/2015 14:18:13,23/02/2015 14:18:14,2,1
3,PAUL,23/02/2015 14:27:52,23/02/2015 14:29:55,2,1
4,LEWIS,26/02/2015 11:38:22,26/02/2015 11:39:27,3,1


In [13]:
pickdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159980 entries, 0 to 159979
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   PH_PICKEDB  159966 non-null  object
 1   PH_PICKSTA  159967 non-null  object
 2   PH_PICKEND  159979 non-null  object
 3   PH_TOTALLI  159980 non-null  int64 
 4   PH_TOTALBO  159980 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 7.3+ MB


In [14]:
pickdf.isna().sum()

PH_PICKEDB    14
PH_PICKSTA    13
PH_PICKEND     1
PH_TOTALLI     0
PH_TOTALBO     0
dtype: int64

In [15]:
# this is a very small # of nulls, will drop all
pickdf.dropna(inplace=True)

In [16]:
pickdf.isna().sum()

PH_PICKEDB    0
PH_PICKSTA    0
PH_PICKEND    0
PH_TOTALLI    0
PH_TOTALBO    0
dtype: int64

In [17]:
pickdf = pickdf.rename(columns={'PH_PICKEDB': 'operator', 'PH_PICKSTA': 'start_time', 'PH_PICKEND': 'end_time', 'PH_TOTALLI': 'total_lines', 'PH_TOTALBO': 'total_boxes'})

In [18]:
pickdf.head()

,operator,start_time,end_time,total_lines,total_boxes
0,PAUL,23/02/2015 12:25:47,23/02/2015 12:25:51,2,1
1,PAUL,23/02/2015 13:48:03,23/02/2015 13:48:05,1,1
2,PAUL,23/02/2015 14:18:13,23/02/2015 14:18:14,2,1
3,PAUL,23/02/2015 14:27:52,23/02/2015 14:29:55,2,1
4,LEWIS,26/02/2015 11:38:22,26/02/2015 11:39:27,3,1


In [19]:
# convert start_time and end_time to date time
pickdf['start']= pd.to_datetime(pickdf['start_time'])
pickdf['end']= pd.to_datetime(pickdf['end_time'])

In [20]:
pickdf.head()

,operator,start_time,end_time,total_lines,total_boxes,start,end
0,PAUL,23/02/2015 12:25:47,23/02/2015 12:25:51,2,1,2015-02-23 12:25:47,2015-02-23 12:25:51
1,PAUL,23/02/2015 13:48:03,23/02/2015 13:48:05,1,1,2015-02-23 13:48:03,2015-02-23 13:48:05
2,PAUL,23/02/2015 14:18:13,23/02/2015 14:18:14,2,1,2015-02-23 14:18:13,2015-02-23 14:18:14
3,PAUL,23/02/2015 14:27:52,23/02/2015 14:29:55,2,1,2015-02-23 14:27:52,2015-02-23 14:29:55
4,LEWIS,26/02/2015 11:38:22,26/02/2015 11:39:27,3,1,2015-02-26 11:38:22,2015-02-26 11:39:27


In [21]:
pickdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159966 entries, 0 to 159979
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   operator     159966 non-null  object        
 1   start_time   159966 non-null  object        
 2   end_time     159966 non-null  object        
 3   total_lines  159966 non-null  int64         
 4   total_boxes  159966 non-null  int64         
 5   start        159966 non-null  datetime64[ns]
 6   end          159966 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 9.8+ MB


In [22]:
pickdf = pickdf.drop(columns=['start_time', 'end_time'])

# START HERE

### For 1st iteration looking to create baseline and model to predict boxes/hr that beats baseline

#### for this iteration total lines will be defined as the number of line items on the order

In [23]:
# Wrangling
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import acquire
import prepare
import wrangle_pick

# Exploring
import scipy.stats as stats

# Visualizing
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
from sklearn.model_selection import learning_curve

pd.options.display.float_format = '{:20,.2f}'.format

### more data cleaning needed   

- sort operator column into actual operators
    - for now remove all operator names that occur less than 10 times
    - a list of those has been made as variable one_off if needed later
    - one_off operators will need to be removed from full dataframe before modeling
- add pick time in seconds column
- removed all from 2015 and 2020
   - vis shows 2015 and 2020 significantly lower volume, for continuity with work with years 2016-2019 where volume is in consistent range
- add lines per box
- add sec per box
- add sec per line
- drop single observation with negative pick time

### completed and added to prepare function

In [24]:
train, test, validate = wrangle_pick.wrangle_pick_data()
train.shape, test.shape, validate.shape

Acquire: downloading raw data files...
Acquire: Completed!
Prepare: Cleaning acquired data...
Prepare: Completed!


((96644, 18), (20065, 18), (17055, 18))

In [25]:
train.to_csv('train_v1.csv')

In [26]:
train.head()

,operator,total_lines,total_boxes,start,end,pick_time,pick_seconds,int_day,day_name,start_year,start_month,start_Y_M,end_year,end_month,end_Y_M,sec_per_box,lines_per_box,sec_per_line
50058,IT,3,1,2016-11-10 09:58:58,2016-11-10 10:00:57,0 days 00:01:59,119.00,3,Thursday,2016,11,2016-11,2016,11,2016-11,119.00,3.00,39.67
78958,IT,71,1,2017-08-22 08:40:42,2017-08-22 09:08:29,0 days 00:27:47,"1,667.00",1,Tuesday,2017,8,2017-08,2017,8,2017-08,"1,667.00",71.00,23.48
72775,JS,3,1,2017-06-19 16:54:16,2017-06-19 16:58:10,0 days 00:03:54,234.00,0,Monday,2017,6,2017-06,2017,6,2017-06,234.00,3.00,78.00
68159,IT,4,1,2017-04-27 16:51:23,2017-04-27 16:57:45,0 days 00:06:22,382.00,3,Thursday,2017,4,2017-04,2017,4,2017-04,382.00,4.00,95.50
96678,HB,2,1,2018-03-15 12:17:11,2018-03-15 12:17:35,0 days 00:00:24,24.00,3,Thursday,2018,3,2018-03,2018,3,2018-03,24.00,2.00,12.00


In [27]:
train.groupby('operator')[['pick_seconds']].mean()

,pick_seconds
operator,
AH,140.89
ANDREA,170.57
AS,143.36
AW,214.00
CB,143.40
...,...
TS,157.93
UN3090LABE,139.20
W100S/19,259.56


In [28]:
train.groupby('day_name')[['pick_seconds']].mean()

,pick_seconds
day_name,
Friday,144.91
Monday,138.65
Saturday,152.95
Sunday,148.58
Thursday,150.06
Tuesday,148.32
Wednesday,150.54


In [29]:
train.groupby(['day_name', 'operator']).pick_seconds.agg(['mean', 'median', 'min', 'max', 'count'])

mean               median  \
day_name  operator                                               
Friday    AH                       138.73                81.50   
          ANDREA                   158.40                88.00   
          AS                       122.99                65.00   
          CB                       160.86                70.00   
          DACE                     110.34                52.00   
...                                   ...                  ...   
Wednesday SJ                       276.44               162.00   
          SW10G                    125.50                86.50   
          TH                       156.52                88.00   
          TS                       100.12                48.00   
          UN3090LABE               101.33                63.00   

                                      min                  max  count  
day_name  operator                                                     
Friday    AH                         5.00             2,379.00   1108  
          ANDREA                    21.00             2,034.00    378  
          AS                         5.00             1,105.00    233  
          CB                        29.00             1,020.00    159  
          DACE                       4.00             1,827.00    861  
...                                   ...                  ...    ...  
Wednesday SJ                        14.00             1,456.00     43  
          SW10G                     25.00               458.00     12  
          TH                        11.00             2,843.00    437  
          TS                        28.00               690.00     25  
          UN3090LABE                33.00               309.00      6  

[261 rows x 5 columns]

In [30]:
train.groupby('operator').pick_seconds.agg(['mean', 'median', 'min', 'max', 'count'])

,mean,median,min,max,count
operator,,,,,
AH,140.89,79.00,2.00,"2,379.00",7430
ANDREA,170.57,88.00,12.00,"4,503.00",1779
AS,143.36,75.00,5.00,"1,716.00",1161
AW,214.00,101.00,47.00,"1,360.00",22
CB,143.40,75.00,5.00,"1,738.00",949
...,...,...,...,...,...
TS,157.93,58.00,24.00,"2,759.00",122
UN3090LABE,139.20,100.00,33.00,493.00,20
W100S/19,259.56,165.00,42.00,"1,080.00",9


# return to this - 

In [31]:
train[train.operator == 'EDYTA'].start.min()

Timestamp('2016-01-18 09:28:56')

In [32]:
# additional info needed  operator tenure (in this dataset)
op_max_start = train.groupby('operator')['start'].min()
op_max_start
#train.groupby('operator')['end'].max()

operator
AH           2016-01-12 09:46:41
ANDREA       2016-01-09 10:21:41
AS           2016-01-11 09:48:23
AW           2017-07-08 08:59:00
CB           2019-01-08 09:00:04
                     ...        
TS           2017-04-08 11:27:08
UN3090LABE   2017-03-11 09:10:21
W100S/19     2018-11-01 08:33:29
W109/24      2016-04-03 14:41:46
WH109/18     2016-07-07 09:34:46
Name: start, Length: 66, dtype: datetime64[ns]

In [33]:
EDYTA = train[train.operator == 'EDYTA']
EDYTA.sort_values('start')

,operator,total_lines,total_boxes,start,end,pick_time,pick_seconds,int_day,day_name,start_year,start_month,start_Y_M,end_year,end_month,end_Y_M,sec_per_box,lines_per_box,sec_per_line
23921,EDYTA,2,1,2016-01-18 09:28:56,2016-01-18 09:29:01,0 days 00:00:05,5.00,0,Monday,2016,1,2016-01,2016,1,2016-01,5.00,2.00,2.50
23922,EDYTA,2,1,2016-01-18 09:29:21,2016-01-18 09:30:12,0 days 00:00:51,51.00,0,Monday,2016,1,2016-01,2016,1,2016-01,51.00,2.00,25.50
23924,EDYTA,2,1,2016-01-18 09:31:04,2016-01-18 09:31:40,0 days 00:00:36,36.00,0,Monday,2016,1,2016-01,2016,1,2016-01,36.00,2.00,18.00
23925,EDYTA,8,1,2016-01-18 09:32:02,2016-01-18 09:34:03,0 days 00:02:01,121.00,0,Monday,2016,1,2016-01,2016,1,2016-01,121.00,8.00,15.13
23926,EDYTA,5,1,2016-01-18 09:35:11,2016-01-18 09:36:27,0 days 00:01:16,76.00,0,Monday,2016,1,2016-01,2016,1,2016-01,76.00,5.00,15.20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22995,EDYTA,10,2,2016-06-01 14:33:43,2016-06-01 14:39:54,0 days 00:06:11,371.00,2,Wednesday,2016,6,2016-06,2016,6,2016-06,185.50,5.00,37.10
23000,EDYTA,2,1,2016-06-01 14:46:29,2016-06-01 14:47:01,0 days 00:00:32,32.00,2,Wednesday,2016,6,2016-06,2016,6,2016-06,32.00,2.00,16.00
23003,EDYTA,8,1,2016-06-01 14:47:22,2016-06-01 14:49:51,0 days 00:02:29,149.00,2,Wednesday,2016,6,2016-06,2016,6,2016-06,149.00,8.00,18.62
23004,EDYTA,2,1,2016-06-01 14:50:15,2016-06-01 14:51:18,0 days 00:01:03,63.00,2,Wednesday,2016,6,2016-06,2016,6,2016-06,63.00,2.00,31.50


In [34]:
train.head()

,operator,total_lines,total_boxes,start,end,pick_time,pick_seconds,int_day,day_name,start_year,start_month,start_Y_M,end_year,end_month,end_Y_M,sec_per_box,lines_per_box,sec_per_line
50058,IT,3,1,2016-11-10 09:58:58,2016-11-10 10:00:57,0 days 00:01:59,119.00,3,Thursday,2016,11,2016-11,2016,11,2016-11,119.00,3.00,39.67
78958,IT,71,1,2017-08-22 08:40:42,2017-08-22 09:08:29,0 days 00:27:47,"1,667.00",1,Tuesday,2017,8,2017-08,2017,8,2017-08,"1,667.00",71.00,23.48
72775,JS,3,1,2017-06-19 16:54:16,2017-06-19 16:58:10,0 days 00:03:54,234.00,0,Monday,2017,6,2017-06,2017,6,2017-06,234.00,3.00,78.00
68159,IT,4,1,2017-04-27 16:51:23,2017-04-27 16:57:45,0 days 00:06:22,382.00,3,Thursday,2017,4,2017-04,2017,4,2017-04,382.00,4.00,95.50
96678,HB,2,1,2018-03-15 12:17:11,2018-03-15 12:17:35,0 days 00:00:24,24.00,3,Thursday,2018,3,2018-03,2018,3,2018-03,24.00,2.00,12.00


questions for explore
1. total min/max pick time? by operator? by day?
2. does longer tenure = faster pick_time
3. does pick_time average vary significantly by day of week
4. days with most orders? fewest? is there a change in # of operators on fewest vs highest order days?
    - initial visual shows significant drop in box count on Sat and Sun, likely there are fewer shifts on those days

additional variations
- assign some operators are PT (4hr vs 8hr shift)
- define each line as 1 unique item in 1 unique location, based on that infer size of items